In [ ]:
#This script wrangles the data

In [13]:
import pandas as pd
from termcolor import colored
import os
import numpy as np
import glob
import cv2
import math
collectionDir = '../Data/Collections'

In [14]:
# List of videos to process
def getVideosList(moviesLoc):
    path = moviesLoc
    vlist = [f for f in os.listdir(path) if f.endswith('.mp4')]
    return(vlist)

In [15]:
def getFrame(meas,video,outputFolder,frameFreq,eachSec):
    
    counterBouncer = 0

    count = snapCount = 0
    sec = 1
    videoFile = video
    imagesFolder = outputFolder
    cap = cv2.VideoCapture(videoFile)
    
    
    frameRate = int(cap.get(5))
    length  = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)/frameRate)
    
    frac, whole = math.modf(length)
    length = int(whole)
    
    frameEachSec = int(frameRate/frameFreq)
    if frameEachSec == 0:
        frameEachSec = 1 # ex: 10/24 will doesntwork
    if eachSec == True:
        frameFreq = frameRate

    cap.set(1,frameFreq)
     #get framerate property
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number 
        ret, frame = cap.read()
        if (ret != True):
            break
            
        if (count % frameFreq == 0):
            
            filename = imagesFolder + "/"+meas+"_" +  str(int(snapCount)) + ".jpg"
            #print('frame:{fr}'.format(fr=frameId))
            cv2.imwrite(filename, frame)
            snapCount += 1
            if snapCount % frameEachSec == 0:
                #print('-----------Second{secc}'.format(secc=sec))
                sec += 1
            if sec == length+1:
                break
        count += 1
      
    cap.release()
    return(snapCount,length,frameRate,frameFreq)

In [16]:
def processVideos(dirr,frameFreq,eachSec):
    
    moviesLoc = collectionDir+'/'+dirr+'/sized/'
    dumpLoc = collectionDir+'/'+dirr+'/Dump/dumpData.csv'

    processed = []
    vlist = getVideosList(moviesLoc)
    
    if len(vlist) > 0:
        outDump = pd.DataFrame()
        #loading dump

        dump = pd.read_csv(dumpLoc,encoding = "ISO-8859-1")
        dump.reset_index()
        dump = dump.apply (pd.to_numeric, errors='coerce')
        
        outFolder = '../Images'
        try:
            os.mkdir(outFolder)
            print("Directory " , outFolder ,  " Created ") 
        except FileExistsError:
            print("Directory " , outFolder ,  " already exists.")
    
        for video in vlist:
           
            if video not in processed:
                
                print('##Processing video: {vv}'.format(vv=video))
                import pdb
                #pdb.set_trace()
                measpoint = video[:-4]
                
                try:
                    a = int(measpoint)
                except:
                    print(colored('Invalid Measpoint','red'))
                    continue
                
                is_measpoint = dump.loc[dump['Measpoint'] == int(measpoint)]
                if is_measpoint.shape[0] < 1:
                    print(colored('Cannot find {mp} in dump of {drr}'.format(mp=measpoint,drr=dirr),'red'))
                    continue
                
                # capture frame with frequency frameFreq  
                vidAddr = moviesLoc+video
                snapCount = None
                snapCount,length,frameRate,frameFreq = getFrame(measpoint,vidAddr,outFolder,frameFreq,eachSec)
                measStart = dump[dump.Measpoint == int(measpoint)].index[0]
                measEnd = int(measStart+(length))
                
                measDump = dump[measStart:measEnd]
                
                #duplicate rows frameFreq times
                measDump = measDump.loc[measDump.index.repeat(int(frameRate/frameFreq))].reset_index(drop=True)
                
                if snapCount == measDump.shape[0]:
                    snapColr = 'green'
                else:
                                        
                    measDump.drop(measDump.tail(1).index,inplace=True)
                    if snapCount != measDump.shape[0]:
                        snapColr ='red'
                    
                print(colored("\t--generated {snp} snaps and {dp} datapoints".format(snp=snapCount,dp=measDump.shape[0]),snapColr))
                
                
                #replace any 0.00, if present
                measDump.loc[measDump.Measpoint != int(measpoint), 'Measpoint'] = int(measpoint)
                imageNumbs = np.arange(0, snapCount, 1).tolist()
                imageNumbs = [str(measpoint) +'_'+str(x)  for x in imageNumbs]
                measDump['Image'] = imageNumbs
                
                outDump = pd.concat([outDump,measDump])
                
                processed.append(video)
            else:
                print("Video {v} already processed".format(v=video))
        outDump.to_csv('../CSV/flameData.csv')
    else:
        print("No videos found in {dr}".format(dr=dirr))
    

In [17]:
def processCollection(collectionDir,frameFreq,eachSec):

    dataDirs = os.listdir(collectionDir)
    
    for dirr in dataDirs:
        print('\n####Processing {d} ...'.format(d=dirr))
        processVideos(dirr,frameFreq,eachSec)
   

In [18]:
processCollection(collectionDir,frameFreq=100,eachSec=True)


####Processing Data_2013-02-20 ...
Directory  ../Images  already exists.
##Processing video: 3069.mp4
	--generated 44 snaps and 44 datapoints
##Processing video: 3070.mp4
	--generated 48 snaps and 48 datapoints
##Processing video: 3071.mp4
	--generated 48 snaps and 48 datapoints
##Processing video: 3072.mp4
	--generated 47 snaps and 47 datapoints
##Processing video: 3073.mp4
	--generated 49 snaps and 49 datapoints
##Processing video: 3074.mp4
	--generated 274 snaps and 274 datapoints
##Processing video: 3075.mp4
	--generated 37 snaps and 37 datapoints
##Processing video: 3076.mp4
	--generated 43 snaps and 43 datapoints
##Processing video: 3077.mp4
	--generated 39 snaps and 39 datapoints
##Processing video: 3078.mp4
	--generated 76 snaps and 76 datapoints
##Processing video: 3079.mp4
	--generated 45 snaps and 45 datapoints
##Processing video: 3080.mp4
	--generated 59 snaps and 59 datapoints
##Processing video: 3081.mp4
	--generated 42 snaps and 42 datapoints
##Processing video: 3082.mp

	--generated 48 snaps and 48 datapoints
##Processing video: 3127.mp4
	--generated 49 snaps and 49 datapoints
##Processing video: 3128.mp4
	--generated 22 snaps and 22 datapoints
##Processing video: 3129.mp4
	--generated 23 snaps and 23 datapoints
##Processing video: 3130.mp4
	--generated 21 snaps and 21 datapoints
##Processing video: 3131.mp4
	--generated 31 snaps and 31 datapoints
##Processing video: 3132.mp4
	--generated 31 snaps and 31 datapoints
##Processing video: 3133.mp4
	--generated 35 snaps and 35 datapoints
##Processing video: 3134.mp4
	--generated 41 snaps and 41 datapoints
##Processing video: 3135.mp4
	--generated 34 snaps and 34 datapoints
##Processing video: 3136.mp4
	--generated 38 snaps and 38 datapoints
##Processing video: 3137.mp4
	--generated 26 snaps and 26 datapoints
##Processing video: 3138.mp4
	--generated 30 snaps and 30 datapoints
##Processing video: 3139.mp4
	--generated 31 snaps and 31 datapoints
##Processing video: 3140.mp4
	--generated 38 snaps and 38 datap

	--generated 33 snaps and 33 datapoints
##Processing video: 3233.mp4
	--generated 33 snaps and 33 datapoints
##Processing video: 3234.mp4
	--generated 31 snaps and 31 datapoints
##Processing video: 3235.mp4
	--generated 32 snaps and 32 datapoints
##Processing video: 3236.mp4
	--generated 31 snaps and 31 datapoints
##Processing video: 3237.mp4
	--generated 32 snaps and 32 datapoints
##Processing video: 3238.mp4
	--generated 30 snaps and 30 datapoints
##Processing video: 3239.mp4
	--generated 31 snaps and 31 datapoints
##Processing video: 3240.mp4
	--generated 32 snaps and 32 datapoints
##Processing video: 3241.mp4
	--generated 33 snaps and 33 datapoints
##Processing video: 3242.mp4
	--generated 32 snaps and 32 datapoints
##Processing video: 3243.mp4
	--generated 31 snaps and 31 datapoints
##Processing video: 3244.mp4
	--generated 31 snaps and 31 datapoints
##Processing video: 3245.mp4
	--generated 31 snaps and 31 datapoints
##Processing video: 3246.mp4
	--generated 32 snaps and 32 datap

	--generated 49 snaps and 49 datapoints
##Processing video: 3342.mp4
	--generated 45 snaps and 45 datapoints
##Processing video: 3343.mp4
	--generated 47 snaps and 47 datapoints
##Processing video: 3344.mp4
	--generated 57 snaps and 57 datapoints
##Processing video: 3345.mp4
	--generated 46 snaps and 46 datapoints
##Processing video: 3346.mp4
	--generated 46 snaps and 46 datapoints
##Processing video: 3347.mp4
	--generated 40 snaps and 40 datapoints
##Processing video: 3348.mp4
	--generated 36 snaps and 36 datapoints
##Processing video: 3349.mp4
	--generated 41 snaps and 41 datapoints
##Processing video: 3350.mp4
	--generated 32 snaps and 32 datapoints
##Processing video: 3351.mp4
	--generated 32 snaps and 32 datapoints
##Processing video: 3352.mp4
	--generated 33 snaps and 33 datapoints
##Processing video: 3353.mp4
	--generated 32 snaps and 32 datapoints
##Processing video: 3354.mp4
	--generated 32 snaps and 32 datapoints
##Processing video: 3355.mp4
	--generated 35 snaps and 35 datap